# Context-Aware Kana-Kanji Converter (v3.1)

**Supports**: Google Colab & Kaggle

**Key Features**:
- Uses zenz dataset **directly** (no tokenization needed)
- **Length-based bucketing** for stable GRU training
- Bidirectional GRU + Attention (best for this task)
- **Masked loss** (ignores PAD tokens for real accuracy)
- All cache saved to **drive** for fast re-runs
- **Memory-safe**: releases data after each processing step

**Input Format**: `context<SEP>kana` → `kanji`

**v3.1 Fixes**: Output truncation, masked loss, gradient clipping, shared encode

In [ ]:
import os
import gc

# Auto-detect platform (Colab check first - Colab also has /kaggle dir!)
if 'COLAB_RELEASE_TAG' in os.environ:
    PLATFORM = 'Colab'
    from google.colab import drive
    drive.mount('/content/drive')
    DRIVE_DIR = '/content/drive/MyDrive/Keyboard-Suggestions-ML-Colab'
elif os.path.exists('/kaggle/working'):
    PLATFORM = 'Kaggle'
    DRIVE_DIR = '/kaggle/working'
else:
    PLATFORM = 'Local'
    DRIVE_DIR = './output'

MODEL_DIR = f"{DRIVE_DIR}/models/gru_japanese_kana_kanji"
CACHE_DIR = f"{DRIVE_DIR}/cache/kana_kanji"  # Saved to drive for persistence!
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(CACHE_DIR, exist_ok=True)

print(f"✅ Platform: {PLATFORM}")
print(f"📁 Model: {MODEL_DIR}")
print(f"💾 Cache: {CACHE_DIR} (persisted to drive)")

In [ ]:
!pip install -q tensorflow keras datasets numpy tqdm

In [ ]:
# ===========================================================
# CONFIGURATION
# ===========================================================
TESTING_MODE = False
MAX_SAMPLES = 8_000_000  # Start with 8M (set None for all ~17.5M)
BATCH_SIZE = 512
FORCE_REBUILD_CACHE = False

NUM_EPOCHS = 3 if TESTING_MODE else 10

# Length limits (filter long sequences)
MAX_CONTEXT_LEN = 30   # left_context max chars
MAX_INPUT_LEN = 30     # kana input max chars
MAX_OUTPUT_LEN = 30    # kanji output max chars
MAX_ENCODER_LEN = MAX_CONTEXT_LEN + 1 + MAX_INPUT_LEN  # context + <SEP>(1 token) + input
MAX_DECODER_LEN = MAX_OUTPUT_LEN + 2  # BOS + content + EOS (FIX: +2 not +1)

# Architecture (controls model size)
CHAR_VOCAB_SIZE = 6000
EMBEDDING_DIM = 64
GRU_UNITS = 128
NUM_ENCODER_LAYERS = 2
NUM_DECODER_LAYERS = 2

SPECIAL_TOKENS = ['<PAD>', '<UNK>', '<BOS>', '<EOS>', '<SEP>']
PAD_IDX = 0  # <PAD> is always index 0
SEP_TOKEN = '<SEP>'

print(f"Encoder max: {MAX_ENCODER_LEN}, Decoder max: {MAX_DECODER_LEN}")

## 0. Shared Utilities

Single encode function used by both training and inference.

In [ ]:
def tokenize_with_sep(text):
    """Tokenize text handling <SEP> as single token."""
    tokens = []
    i = 0
    while i < len(text):
        if text[i:i+5] == '<SEP>':
            tokens.append('<SEP>')
            i += 5
        else:
            tokens.append(text[i])
            i += 1
    return tokens

def encode_tokens(tokens, vocab, max_len, pad_id, unk_id):
    """Encode token list to padded integer IDs."""
    ids = [vocab.get(t, unk_id) for t in tokens][:max_len]
    return ids + [pad_id] * (max_len - len(ids))

def encode_encoder_input(text, vocab, pad_id, unk_id):
    """Encode encoder input (context<SEP>kana)."""
    tokens = tokenize_with_sep(text)
    return encode_tokens(tokens, vocab, MAX_ENCODER_LEN, pad_id, unk_id)

def encode_decoder_seq(text, vocab, pad_id, unk_id, add_bos=False, add_eos=False):
    """Encode decoder sequence with optional BOS/EOS."""
    tokens = []
    if add_bos:
        tokens.append('<BOS>')
    tokens.extend(list(text))
    if add_eos:
        tokens.append('<EOS>')
    return encode_tokens(tokens, vocab, MAX_DECODER_LEN, pad_id, unk_id)

print("✓ Shared utilities loaded")

## 1. Load or Build Cached Data

Uses zenz dataset **directly** - no tokenization needed!

**Cache is saved to drive** - next run will be fast!

In [ ]:
import json
import numpy as np
from tqdm.auto import tqdm

VOCAB_CACHE = f"{CACHE_DIR}/vocabulary_v3.json"
TENSORS_CACHE = f"{CACHE_DIR}/tensors_v3_1.npz"  # v3.1 (new decoder length)

def cache_exists():
    return os.path.exists(VOCAB_CACHE) and os.path.exists(TENSORS_CACHE)

if cache_exists() and not FORCE_REBUILD_CACHE:
    print("📦 Loading from cache (saved in drive)...")
    
    with open(VOCAB_CACHE, 'r', encoding='utf-8') as f:
        vocab_data = json.load(f)
    char_to_idx = vocab_data['char_to_idx']
    idx_to_char = {int(k): v for k, v in vocab_data['idx_to_char'].items()}
    vocab_size = len(char_to_idx)
    
    tensors = np.load(TENSORS_CACHE)
    encoder_inputs = tensors['encoder_inputs']
    decoder_inputs = tensors['decoder_inputs']
    decoder_targets = tensors['decoder_targets']
    
    print(f"✓ Loaded {len(encoder_inputs):,} samples")
    print(f"✓ Encoder shape: {encoder_inputs.shape}, Decoder shape: {decoder_inputs.shape}")
    CACHE_LOADED = True
else:
    print("🔨 Building from scratch (will save to drive for next time)...")
    CACHE_LOADED = False

In [ ]:
# Load dataset directly - no tokenization needed!
if not CACHE_LOADED:
    from datasets import load_dataset
    
    print("📥 Loading zenz dataset...")
    dataset = load_dataset(
        "Miwa-Keita/zenz-v2.5-dataset",
        data_files="train_wikipedia.jsonl",
        split="train"
    )
    print(f"✓ Raw dataset: {len(dataset):,} items")

In [ ]:
# Filter and prepare training data (fast - no tokenization!)
if not CACHE_LOADED:
    print("\n🔍 Filtering and preparing data...")
    
    training_data = []
    skipped = {'too_long': 0, 'empty': 0}
    
    for item in tqdm(dataset, desc="Processing"):
        # Get fields from dataset
        kana_input = item.get('input', '') or ''  # Katakana input
        kanji_output = item.get('output', '') or ''  # Kanji output
        left_context = item.get('left_context', '') or ''  # Context (can be null)
        
        # Skip empty
        if not kana_input or not kanji_output:
            skipped['empty'] += 1
            continue
        
        # Skip too long
        if (len(left_context) > MAX_CONTEXT_LEN or 
            len(kana_input) > MAX_INPUT_LEN or 
            len(kanji_output) > MAX_OUTPUT_LEN):
            skipped['too_long'] += 1
            continue
        
        # Format: "context<SEP>kana" -> "kanji"
        encoder_text = f"{left_context}<SEP>{kana_input}"
        decoder_text = kanji_output
        
        training_data.append({
            'input': encoder_text,
            'output': decoder_text,
            'input_len': len(kana_input)  # For bucketing
        })
        
        # Stop if reached limit
        if MAX_SAMPLES and len(training_data) >= MAX_SAMPLES:
            break
    
    print(f"\n✓ Valid examples: {len(training_data):,}")
    print(f"  Skipped (too long): {skipped['too_long']:,}")
    print(f"  Skipped (empty): {skipped['empty']:,}")
    
    # 🧹 Release HuggingFace dataset (~1-2 GB)
    del dataset, skipped
    gc.collect()
    print("🧹 Released dataset from memory")

In [ ]:
# Sort by length for bucketing (helps GRU training stability)
if not CACHE_LOADED:
    print("\n📊 Sorting by length (bucketing)...")
    training_data.sort(key=lambda x: x['input_len'])
    
    # Show length distribution
    lengths = [d['input_len'] for d in training_data]
    print(f"  Short (0-10): {sum(1 for l in lengths if l <= 10):,}")
    print(f"  Medium (11-20): {sum(1 for l in lengths if 10 < l <= 20):,}")
    print(f"  Long (21+): {sum(1 for l in lengths if l > 20):,}")
    del lengths

In [ ]:
# Build vocabulary
if not CACHE_LOADED:
    from collections import Counter
    
    print("\n📝 Building vocabulary...")
    char_counts = Counter()
    
    for d in tqdm(training_data, desc="Counting chars"):
        # Count chars (excluding <SEP> marker)
        text = d['input'].replace('<SEP>', '') + d['output']
        char_counts.update(list(text))
    
    # Build vocab with special tokens first
    char_to_idx = {tok: i for i, tok in enumerate(SPECIAL_TOKENS)}
    for char, _ in char_counts.most_common(CHAR_VOCAB_SIZE - len(SPECIAL_TOKENS)):
        char_to_idx[char] = len(char_to_idx)
    
    idx_to_char = {v: k for k, v in char_to_idx.items()}
    vocab_size = len(char_to_idx)
    print(f"✓ Vocab size: {vocab_size}")
    
    # 🧹 Release char_counts
    del char_counts
    gc.collect()
    print("🧹 Released char_counts from memory")

In [ ]:
# Encode to tensors (using shared encode functions)
if not CACHE_LOADED:
    print("\n🔢 Encoding to tensors...")
    
    PAD = char_to_idx['<PAD>']
    UNK = char_to_idx['<UNK>']
    
    # Encode all data using shared utility functions
    encoder_inputs = np.array(
        [encode_encoder_input(d['input'], char_to_idx, PAD, UNK) for d in tqdm(training_data, desc="Enc input")],
        dtype=np.int32
    )
    decoder_inputs = np.array(
        [encode_decoder_seq(d['output'], char_to_idx, PAD, UNK, add_bos=True) for d in tqdm(training_data, desc="Dec input")],
        dtype=np.int32
    )
    decoder_targets = np.array(
        [encode_decoder_seq(d['output'], char_to_idx, PAD, UNK, add_eos=True) for d in tqdm(training_data, desc="Dec target")],
        dtype=np.int32
    )
    
    # 🧹 Release training_data (~2-3 GB of dicts/strings)
    del training_data
    gc.collect()
    print("🧹 Released training_data from memory")
    
    print(f"\n✓ Shapes: enc={encoder_inputs.shape}, dec_in={decoder_inputs.shape}, dec_tgt={decoder_targets.shape}")
    
    # Verify: decoder_inputs[0] should start with BOS, decoder_targets[0] should end with EOS
    bos_id = char_to_idx['<BOS>']
    eos_id = char_to_idx['<EOS>']
    assert decoder_inputs[0][0] == bos_id, f"First decoder input should be BOS, got {decoder_inputs[0][0]}"
    # Find EOS in target
    target_list = list(decoder_targets[0])
    assert eos_id in target_list, f"Decoder target should contain EOS"
    print("✓ BOS/EOS verification passed")
    
    # Save cache to drive
    print("\n💾 Saving cache to drive (for next run)...")
    with open(VOCAB_CACHE, 'w', encoding='utf-8') as f:
        json.dump({
            'char_to_idx': char_to_idx,
            'idx_to_char': {str(k): v for k, v in idx_to_char.items()}
        }, f, ensure_ascii=False)
    
    np.savez_compressed(
        TENSORS_CACHE,
        encoder_inputs=encoder_inputs,
        decoder_inputs=decoder_inputs,
        decoder_targets=decoder_targets
    )
    print(f"✓ Cached to {CACHE_DIR}")

print(f"\n📊 Data: {encoder_inputs.shape[0]:,} samples")
print(f"   Encoder: {encoder_inputs.shape}, Decoder: {decoder_inputs.shape}")

## 2. Create Dataset

In [ ]:
import tensorflow as tf

# Shuffle within buckets (keep similar lengths together)
def bucket_shuffle(data, bucket_size=50000):
    """Shuffle within buckets to maintain length grouping."""
    n = len(data)
    indices = []
    for start in range(0, n, bucket_size):
        end = min(start + bucket_size, n)
        bucket_idx = list(range(start, end))
        np.random.shuffle(bucket_idx)
        indices.extend(bucket_idx)
    return np.array(indices)

idx = bucket_shuffle(encoder_inputs)
encoder_inputs = encoder_inputs[idx]
decoder_inputs = decoder_inputs[idx]
decoder_targets = decoder_targets[idx]

# Split
split = int(len(encoder_inputs) * 0.9)
train_ds = tf.data.Dataset.from_tensor_slices((
    {'encoder_input': encoder_inputs[:split], 'decoder_input': decoder_inputs[:split]},
    decoder_targets[:split]
)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices((
    {'encoder_input': encoder_inputs[split:], 'decoder_input': decoder_inputs[split:]},
    decoder_targets[split:]
)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

print(f"Train: {split:,}, Val: {len(encoder_inputs)-split:,}")

# 🧹 Release numpy arrays (tf.data has its own copy)
del encoder_inputs, decoder_inputs, decoder_targets, idx
gc.collect()
print("🧹 Released numpy arrays (tf.data has copy)")

## 3. Build Model (Bidirectional GRU + Attention)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, GRU, Dense, Dropout, Bidirectional, Attention, Concatenate, LayerNormalization

# Shared embedding
emb = Embedding(vocab_size, EMBEDDING_DIM, mask_zero=False, name='embedding')

# Encoder (Bidirectional GRU - reads input forward and backward)
enc_in = Input(shape=(MAX_ENCODER_LEN,), dtype='int32', name='encoder_input')
x = Dropout(0.1)(emb(enc_in))
for i in range(NUM_ENCODER_LAYERS):
    x = LayerNormalization()(Bidirectional(GRU(GRU_UNITS, return_sequences=True), name=f'enc_{i+1}')(x))
enc_out = x

# Decoder (GRU with Attention) - FIX: shape uses MAX_DECODER_LEN
dec_in = Input(shape=(MAX_DECODER_LEN,), dtype='int32', name='decoder_input')
y = Dropout(0.1)(emb(dec_in))
for i in range(NUM_DECODER_LAYERS):
    y = LayerNormalization()(GRU(GRU_UNITS * 2, return_sequences=True, name=f'dec_{i+1}')(y))

# Attention mechanism
ctx = Attention(use_scale=True, name='attn')([y, enc_out])

# Output
combined = Concatenate()([y, ctx])
combined = LayerNormalization()(combined)
combined = Dropout(0.2)(combined)
combined = Dense(GRU_UNITS * 2, activation='relu')(combined)
out = Dense(vocab_size, activation='softmax', name='output')(combined)

model = Model([enc_in, dec_in], out, name='kana_kanji_v3_1')

# FIX: Masked loss to ignore PAD tokens + gradient clipping
def masked_sparse_ce(y_true, y_pred):
    """Sparse categorical crossentropy that ignores PAD (index 0)."""
    mask = tf.cast(tf.not_equal(y_true, PAD_IDX), tf.float32)
    loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)
    masked_loss = loss * mask
    return tf.reduce_sum(masked_loss) / (tf.reduce_sum(mask) + 1e-8)

def masked_accuracy(y_true, y_pred):
    """Accuracy that ignores PAD tokens (real accuracy)."""
    mask = tf.cast(tf.not_equal(y_true, PAD_IDX), tf.float32)
    pred_ids = tf.cast(tf.argmax(y_pred, axis=-1), y_true.dtype)
    correct = tf.cast(tf.equal(y_true, pred_ids), tf.float32) * mask
    return tf.reduce_sum(correct) / (tf.reduce_sum(mask) + 1e-8)

# FIX: Gradient clipping
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3, clipnorm=1.0)

model.compile(
    optimizer=optimizer,
    loss=masked_sparse_ce,
    metrics=[masked_accuracy]
)

# Show actual model size
actual_params = model.count_params()
print(f"📊 Actual params: {actual_params:,}")
print(f"   FP32 (Keras .keras): ~{actual_params * 4 / 1024 / 1024:.1f} MB")
print(f"   FP16 (TFLite): ~{actual_params * 2 / 1024 / 1024:.1f} MB")
model.summary()

## 4. Train

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

callbacks = [
    ModelCheckpoint(f'{MODEL_DIR}/best_v3_1.keras', monitor='val_masked_accuracy', save_best_only=True, verbose=1),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6, verbose=1)
]

history = model.fit(
    train_ds,
    epochs=NUM_EPOCHS,
    validation_data=val_ds,
    callbacks=callbacks
)

In [ ]:
import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
ax1.plot(history.history['loss'], label='Train')
ax1.plot(history.history['val_loss'], label='Val')
ax1.legend(); ax1.set_title('Loss (masked, ignores PAD)')

ax2.plot(history.history['masked_accuracy'], label='Train')
ax2.plot(history.history['val_masked_accuracy'], label='Val')
ax2.legend(); ax2.set_title('Accuracy (masked, real)')

plt.savefig(f'{MODEL_DIR}/training_v3_1.png')
plt.show()
print(f"Best val accuracy: {max(history.history['val_masked_accuracy'])*100:.2f}%")

## 5. Save

In [ ]:
# Save model and config
model.save(f'{MODEL_DIR}/model.keras')

with open(f'{MODEL_DIR}/char_to_idx.json', 'w', encoding='utf-8') as f:
    json.dump(char_to_idx, f, ensure_ascii=False)

with open(f'{MODEL_DIR}/idx_to_char.json', 'w', encoding='utf-8') as f:
    json.dump({str(k): v for k, v in idx_to_char.items()}, f, ensure_ascii=False)

with open(f'{MODEL_DIR}/config.json', 'w') as f:
    json.dump({
        'vocab_size': vocab_size,
        'max_encoder_len': MAX_ENCODER_LEN,
        'max_decoder_len': MAX_DECODER_LEN,
        'sep_token': SEP_TOKEN,
        'version': 'v3.1'
    }, f)

# Show actual file size
keras_size = os.path.getsize(f'{MODEL_DIR}/model.keras')
print(f"✓ Model saved: {keras_size / 1024 / 1024:.2f} MB")

In [ ]:
# Export TFLite
try:
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
    converter._experimental_lower_tensor_list_ops = False
    
    tflite = converter.convert()
    with open(f'{MODEL_DIR}/model.tflite', 'wb') as f:
        f.write(tflite)
    print(f"✓ model.tflite ({len(tflite)/(1024*1024):.2f}MB)")
    
    # FP16 version
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.target_spec.supported_types = [tf.float16]
    tflite16 = converter.convert()
    with open(f'{MODEL_DIR}/model_fp16.tflite', 'wb') as f:
        f.write(tflite16)
    print(f"✓ model_fp16.tflite ({len(tflite16)/(1024*1024):.2f}MB)")
    
except Exception as e:
    print(f"⚠ TFLite export failed: {e}")

## 6. Verification

In [ ]:
print("="*60)
print("VERIFICATION")
print("="*60)

PAD = char_to_idx['<PAD>']
BOS = char_to_idx['<BOS>']
EOS = char_to_idx['<EOS>']
UNK = char_to_idx['<UNK>']
SEP = char_to_idx['<SEP>']

def convert(context, kana):
    """Convert kana to kanji using context. Uses shared encode function."""
    # Use shared encode function (same as training)
    enc_text = f"{context}<SEP>{kana}"
    enc_ids = encode_encoder_input(enc_text, char_to_idx, PAD, UNK)
    enc_in = np.array([enc_ids], dtype=np.int32)
    
    dec_in = np.zeros((1, MAX_DECODER_LEN), dtype=np.int32)
    dec_in[0, 0] = BOS
    
    result = []
    for i in range(MAX_DECODER_LEN - 1):  # -1 because BOS takes first slot
        pred = model.predict({'encoder_input': enc_in, 'decoder_input': dec_in}, verbose=0)
        next_id = int(np.argmax(pred[0, i]))
        
        if next_id == EOS:
            break
        if next_id not in [PAD, BOS, EOS, UNK, SEP]:
            result.append(idx_to_char.get(next_id, ''))
        
        if i + 1 < MAX_DECODER_LEN:
            dec_in[0, i + 1] = next_id
    
    return ''.join(result)

# Test cases (context, kana, expected kanji)
tests = [
    # Same kana, different kanji based on context
    ("今日はとても", "アツイ", "暑い"),
    ("お茶が", "アツイ", "熱い"),
    ("この辞典は", "アツイ", "厚い"),
    # Speed vs time
    ("毎朝起きるのが", "ハヤイ", "早い"),
    ("彼は走るのが", "ハヤイ", "速い"),
    # Bridge/chopsticks/edge
    ("川に", "ハシ", "橋"),
    ("ご飯を", "ハシ", "箸"),
    ("道の", "ハシ", "端"),
    # Listen/hear
    ("音楽を", "キク", "聴く"),
    ("道を", "キク", "聞く"),
    # Take photo/take salt
    ("写真を", "トル", "撮る"),
    ("塩を", "トル", "取る"),
]

correct = 0
for ctx, kana, expected in tests:
    result = convert(ctx, kana)
    ok = result == expected or expected in result or result in expected
    if ok:
        correct += 1
    print(f"{'✓' if ok else '✗'} {ctx}<SEP>{kana} → {result} (expected: {expected})")

print(f"\n✅ Score: {correct}/{len(tests)} ({correct/len(tests)*100:.0f}%)")

In [ ]:
# List saved files
print(f"\n📦 Files ({PLATFORM}):")
for f in sorted(os.listdir(MODEL_DIR)):
    p = f'{MODEL_DIR}/{f}'
    if os.path.isfile(p):
        s = os.path.getsize(p)
        if s > 1024*1024:
            print(f"  {f}: {s/(1024*1024):.2f}MB")
        else:
            print(f"  {f}: {s/1024:.1f}KB")